In [ ]:
# download dataset using kaggle api
import kaggle
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

In [2]:
# extract file from a zipfile

import zipfile
zip_file = zipfile.ZipFile('orders.csv.zip')
zip_file.extractall()  #extracting zipfile to dir
zip_file.close()   #close file

In [3]:
# read data from the file and handle null values

import pandas as pd

df = pd.read_csv('orders.csv')
#df.head(20)
#df['Ship Mode'].unique()

# in this case we see that we have 'Not Available' and 'Unknown' values in ship mode column and we want to make them null/nan values in the file
# so we are going to add na_values when reading a file in order to handle null values in the file.


In [5]:
# handling null values

df = pd.read_csv('orders.csv', na_values= ['Not Available','unknown'])
df.head(20)
df['Ship Mode'].unique()  #now we have all the null values as nan 

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [12]:
#rename column names.. make them lower case and replace space with underscore

#df.rename(columns={'Order Id': 'order_id', 'City': 'city'})  #but this is not a good practice as there are 15 columns and we need to provide
                                                            #column names and renames each time
df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace(' ','_')


In [7]:
#derive new columns discount, sale price and profit
df['discount'] = df['list_price']*df['discount_percent']*.01
df['sale_price'] = df['list_price'] - df['discount']
df['profit']= df['sale_price'] - df['cost_price']

In [13]:
#convert order date from object data type to datetime
df.dtypes
df['order_date']=pd.to_datetime(df['order_date'], format="%Y-%m-%d")


In [8]:
# drop columns: cost price, list price and discount percent

df.drop(columns=['cost_price','list_price','discount_percent'], inplace=True)

# we have used inplace argument so the change would be reflected permanently 



In [ ]:
#load the data into sql server using replace option
!pip install sqlalchemy pyodbc
import sqlalchemy as sal
engine = sal.create_engine('mssql://LAPTOP-HDFJR04B\SQLEXPRESS/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn=engine.connect()

In [ ]:
# load the data into sql server using append command 

#df.to_sql('df_orders', con=conn, index=False, if_exists= 'replace')
                                                                    
                                                                    # we have used to_sql() in order to load the df dataframe into SQL and 
                                                                   #kept the index=false, it means we don't want index id,
                                                                    # we want the columns starting from order_id, exclude the index column 
#when we used if_exists='replace' option in upper query, it created
#max value of datatypes in sql(bigint, varchar(max) etc) which takes alot of memory so we dropped the df_orders table in sql and we will create a new table there 
#and we will append the data there.

df.to_sql('df_orders', con=conn, index=False, if_exists= 'append')